<a href="https://colab.research.google.com/github/riccardo1980/tf_estimators/blob/master/MNIST_keras_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple CNN for Image Classification
-  dataset: MNIST
- TensorFlow: tf.keras.layers, tf.Estimator
- Colab TensorBoard monitoring

In [0]:
import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [3]:
def make_model():
 
  model = tf.keras.Sequential()

  # Input Layer
  model.add(tf.keras.layers.InputLayer(input_shape=[28*28], name='x'))
  model.add(tf.keras.layers.Reshape(target_shape=[28, 28, 1]))

  # Convolutional Layers
  model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=[5, 5],
                                   padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=[2, 2]))

  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=[5, 5],
                                   padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPool2D(pool_size=[2,2], strides=[2, 2]))

  # Reshape
  model.add(tf.keras.layers.Reshape(target_shape=[7 * 7 * 64]))

  # Dense Layer
  model.add(tf.keras.layers.Dense(units=1024, activation='relu'))
  model.add(tf.keras.layers.Dropout(rate=0.4))

  # Logits Layer
  model.add(tf.keras.layers.Dense(units=10, activation='relu'))

  return model
"""
# Compute predictions
predicted_classes = tf.argmax(input=logits, axis=1)


predictions = {
    'classes': predicted_classes[:, tf.newaxis],
    'probabilities': tf.nn.softmax(logits, name="softmax_tensor"),
    'logits': logits,
}

# PREDICT
if mode == tf.estimator.ModeKeys.PREDICT:
  return tf.estimator.EstimatorSpec(mode=mode,
                                    predictions=predictions)

# Calculate Loss
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,
                                              logits=logits)

# Compute evaluation metrics
accuracy = tf.metrics.accuracy(labels=labels,
                               predictions=predicted_classes,
                               name='acc_op')

metrics = {'accuracy': accuracy}
tf.summary.scalar('accuracy', accuracy[1])

# EVAL
if mode == tf.estimator.ModeKeys.EVAL:
  return tf.estimator.EstimatorSpec(mode,
                                    loss=loss, 
                                    eval_metric_ops=metrics)

# TRAIN
assert mode == tf.estimator.ModeKeys.TRAIN

train_op = optimizer.minimize(loss, 
                              global_step=tf.train.get_global_step())
return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
"""

'\n# Compute predictions\npredicted_classes = tf.argmax(input=logits, axis=1)\n\n\npredictions = {\n    \'classes\': predicted_classes[:, tf.newaxis],\n    \'probabilities\': tf.nn.softmax(logits, name="softmax_tensor"),\n    \'logits\': logits,\n}\n\n# PREDICT\nif mode == tf.estimator.ModeKeys.PREDICT:\n  return tf.estimator.EstimatorSpec(mode=mode,\n                                    predictions=predictions)\n\n# Calculate Loss\nloss = tf.losses.sparse_softmax_cross_entropy(labels=labels,\n                                              logits=logits)\n\n# Compute evaluation metrics\naccuracy = tf.metrics.accuracy(labels=labels,\n                               predictions=predicted_classes,\n                               name=\'acc_op\')\n\nmetrics = {\'accuracy\': accuracy}\ntf.summary.scalar(\'accuracy\', accuracy[1])\n\n# EVAL\nif mode == tf.estimator.ModeKeys.EVAL:\n  return tf.estimator.EstimatorSpec(mode,\n                                    loss=loss, \n                       

In [0]:
def make_train_input_fn(features,
                        labels,
                        batch_size=100,
                        num_epochs=None,
                        shuffle=True):

  train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": features},
                                                      y=labels,
                                                      batch_size=batch_size,
                                                      num_epochs=num_epochs,
                                                      shuffle=shuffle)
  return train_input_fn

def make_eval_input_fn(features,
                       labels):
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": features},
                                                     y=labels,
                                                     num_epochs=1,
                                                     shuffle=False)
  
  return eval_input_fn

In [5]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
MODEL_DIR="/tmp/mnist_convnet_model"
! rm -rf $MODEL_DIR

learning_rate=0.1
batch_size=100
max_steps=20000

In [11]:
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)


def sparse_loss(labels, logits):
    
    #foo = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,
    #                                                     logits=y_pred)
    #return tf.reduce_mean(foo, axis=-1)
    
    return tf.losses.sparse_softmax_cross_entropy(labels,
                                                  logits,
                                                  tf.losses.Reduction.SUM)

model = make_model()
model.compile(optimizer=optimizer,
              loss=sparse_loss,
              metrics=['accuracy'])
model.summary()

TypeError: ignored

In [10]:
# Create the Estimator
my_checkpointing_config = tf.estimator.RunConfig(
    save_checkpoints_steps = 2000,
    #keep_checkpoint_max = 10,
    log_step_count_steps = 500,
    tf_random_seed=2020         # for reproducibility
)


train_spec = tf.estimator.TrainSpec(
    input_fn=make_train_input_fn(train_data,
                                 train_labels,
                                 batch_size=batch_size),
    max_steps=max_steps)

eval_spec = tf.estimator.EvalSpec(
    input_fn=make_eval_input_fn(eval_data,
                                eval_labels),
    steps=None,          # use complete eval set
    start_delay_secs=0,  # start immediately
    throttle_secs=10)    # minimum delay between evaluations


classifier = tf.keras.estimator.model_to_estimator(keras_model=model,
                                                   config=my_checkpointing_config,
                                                   model_dir=MODEL_DIR)

tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': 2020, '_save_summary_steps': 100, '_save_checkpoints_steps': 2000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 500, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcf101ac828>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed

ValueError: ignored

# Get ngrock and run it


In [9]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-02 13:55:01--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.245.79, 52.45.111.123, 52.203.66.95, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.245.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.30M  59.3MB/s    in 0.2s    

2019-05-02 13:55:02 (59.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [10]:
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://75ffbfcf.ngrok.io


# Start TensorBoard

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(MODEL_DIR)
)

# Stop Tensorboard

In [0]:
get_ipython().system_raw('pkill tensorboard')